In [ ]:
import requests
from config import SPOT_CLIENT_SECRET, SPOT_CLIENT_ID

AUTH_URL = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': SPOT_CLIENT_ID,
    'client_secret': SPOT_CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}


In [42]:
import base64
from config import SPOT_CLIENT_SECRET, SPOT_CLIENT_ID
CLIENT_ID = SPOT_CLIENT_ID
CLIENT_SECRET = SPOT_CLIENT_SECRET

def get_spotify_token(client_id, client_secret):
    auth_url = "https://accounts.spotify.com/api/token"

    # Codifica as credenciais no formato Base64
    auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode('utf-8')).decode('utf-8')
    
    headers = {
        "Authorization": f"Basic {auth_header}",
    }
    data = {
        "grant_type": "client_credentials"
    }

    r = requests.post(auth_url, headers=headers, data=data)
    if r.status_code != 200:
        raise Exception(f"Erro ao obter token. Código {r.status_code}: {r.text}")
    
    return r.json()['access_token']

# Usando a função para obter o token
TOKEN = get_spotify_token(CLIENT_ID, CLIENT_SECRET)
headers = {
    "Authorization": f"Bearer {TOKEN}"
}

In [ ]:
BASE_URL = 'https://api.spotify.com/v1/'
track_name = "Bulldog Down in Sunny Tennessee"
artist_name = "Charlie Poole"

# make the API request
r = requests.get(BASE_URL + 'search?q=track:' + track_name + '+artist:' + artist_name + '&type=track', headers=headers)

# get the first track from the results
track = r.json()['tracks']['items'][0]
track_id = track['id']


In [ ]:
r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
audio_features = r.json()


In [ ]:
for feature, value in audio_features.items():
    print(f"{feature}: {value}")

In [ ]:
import pandas as pd

basedir = r"C:\Users\luboh\OneDrive\Área de Trabalho\USP\EngDadosBigData\eEDB-018_REPO\DEVELOPER\eEDB-018\prj\DATA"
data_1 = basedir + r'\panda_dataset_taffc_metadata.csv'

data_source = pd.read_csv(data_1)

#data_source.head()
data_source[['Title','Artist','GenresStr','Quadrant']].head()

In [43]:
import requests
import pandas as pd

# 1. Autenticação (mesmo código anterior)

basedir = r"C:\Users\luboh\OneDrive\Área de Trabalho\USP\EngDadosBigData\eEDB-018_REPO\DEVELOPER\eEDB-018\prj\DATA"
data_1 = basedir + r'\panda_dataset_taffc_metadata.csv'

data_source = pd.read_csv(data_1)
data_source[['Title','Artist','GenresStr','Quadrant']].values

# 2. Lista de faixas (exemplo com as 4 primeiras faixas fornecidas)
tracks = data_source[['Title','Artist','GenresStr','Quadrant']].values
not_found_tracks = []

# 3. Função para buscar características de áudio

def get_audio_features(track_name, artist_name):

    track_name = str(track_name)
    artist_name = str(artist_name)

    r = requests.get(BASE_URL + 'search?q=track:' + track_name + '+artist:' + artist_name + '&type=track', headers=headers)
    response_json = r.json()
    
    # Verificação adicionada aqui
    if 'tracks' not in response_json:
        print(f"Erro na resposta para: {track_name} de {artist_name}. Resposta: {response_json}")
        not_found_tracks.append([track_name, artist_name])
        return None

    items = response_json['tracks']['items']
    
    if not items:
        print(f"Não foi possível encontrar: {track_name} de {artist_name}")
        not_found_tracks.append([track_name, artist_name])
        return None
    
    track = items[0]
    track_id = track['id']
    r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
    return r.json()


# 4. Criar o DataFrame
data = []

for track_name, artist_name, genre, quadrant in tracks:
    audio_features = get_audio_features(track_name, artist_name)

    if audio_features is None:
        continue

    data.append([
        track_name,
        artist_name,
        genre,
        quadrant,
        audio_features['acousticness'],
        audio_features['danceability'],
        audio_features['energy'],
        audio_features['instrumentalness'],
        audio_features['key'],
        audio_features['liveness'],
        audio_features['loudness'],
        audio_features['mode'],
        audio_features['speechiness'],
        audio_features['tempo'],
        audio_features['time_signature'],
        audio_features['valence']
    ])

df = pd.DataFrame(data, columns=[
    'Nome da Faixa',
    'Cantor ou Compositor',
    'Genero',
    'Quadrante',
    'acousticness',
    'danceability',
    'energy',
    'instrumentalness',
    'key',
    'liveness',
    'loudness',
    'mode',
    'speechiness',
    'tempo',
    'time_signature',
    'valence'
])

df_not_found = pd.DataFrame(not_found_tracks, columns=['Nome da Faixa', 'Cantor ou Compositor'])

# 5. Salvar o DataFrame como um arquivo CSV
df.to_csv('spotify_dataset.csv', index=False)
df_not_found.to_csv('not_found_spotify.csv', index=False)

Não foi possível encontrar: Flamencos en el Aire de Gipsy Kings
Não foi possível encontrar: Signal Fire de Danny Elfman
Não foi possível encontrar: My Loved Ones Are Waiting for Me (Waiting and Watching) de The Soul Stirrers
Não foi possível encontrar: Trav'lin' Light de Eddie Heywood and His Orchestra
Não foi possível encontrar: Mami Wata de Olu Dara
Não foi possível encontrar: Polka Dots And Moonbeams de NDR Bigband
Não foi possível encontrar: Ain't No Need of Crying (Every Day Is the Same/Louisiana Blues) de Clifton Chenier
Não foi possível encontrar: God Rest Ye Merry Gentleman de Bing Crosby
Não foi possível encontrar: This Train de The Sam Price Trio
Não foi possível encontrar: Trav'lin' Light de Eddie Heywood
Não foi possível encontrar: Mi Hora Mala Llego (Siguiriyas) de Enrique Morente
Não foi possível encontrar: I've Loved You All Over the World de Willie Nelson
Não foi possível encontrar: Shape of Things to Come de The Black Crowes
Não foi possível encontrar: Long Time Gone d